# Basics


### Quick install instructions
The quick version is basically just
1. `!pip install labelbox`
2. `export LABELBOX_API_KEY="<your_api_key>"`
* Get this from the UI under (Account -> API -> Create API Key)
* You can also set the api_key below in the notebook.

This only works for cloud deployments.
* For more details : https://docs.labelbox.com/python-sdk/en/index-en#labelbox-python-sdk


#### The remainder of this notebook is an interactive version of the fundamental concepts docs.
* For more details you can read the docs here: 
    *    https://docs.labelbox.com/python-sdk/en/index-en#fundamental-concepts

In [1]:
!pip install labelbox

In [2]:
from labelbox import Project, Dataset, Client
from getpass import getpass
import os

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

### Main takeaways:
* All interactions with labelbox happen through the client
* all attributes that are labelbox.orm.Fields can be accessed via object.field_name
* all attributes that are labelbox.orm.Relationships can be accessed via object.relationship()
----
* To use on your own data you need to plug in the following:
1. Project and Dataset ids (go to the web ui and you can find these in the url)
    * (https://app.labelbox.com/projects/<project_id>
    * https://app.labelbox.com/dataset/<dataset_id>
2. A project name and a dataset name
    * Select any project names from here: https://app.labelbox.com/projects
    * Select any dataset names from here: https://app.labelbox.com/data

In [4]:
PROJECT_ID = "ckk4q1viuc0w20704eh69u28h"
DATASET_ID = "ckk4q1vjznyhu087203wlghfr"
PROJECT_NAME = "Sample Project"
DATASET_NAME = "Example Jellyfish Dataset"
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

### Client
* Starting point for all db interactions

In [5]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

In [6]:
#Client can be used to fetch by id:
project = client.get_project(PROJECT_ID)
dataset = client.get_dataset(DATASET_ID)

In [7]:
project

<Project ID: ckk4q1viuc0w20704eh69u28h>

### Fields
* All db objects have fields (look at the source code to see them https://github.com/Labelbox/labelbox-python/blob/develop/labelbox/schema/project.py)
* These fields are attributes of the object

In [8]:
print(project.name)
print(project.description)
print(dataset.name)

Sample Project
Demonstrating image segmentation and object detection
Example Jellyfish Dataset


* Fields can be updated. This will be reflected server side (you will see it in labelbox) 

In [9]:
project.update(description="new description field")
print(project.description)

### Pagination
* Queries that return a list of database objects return them as a PaginatedCollection
* The goal here is to limit the data being returned to only the necessary data.

In [10]:
labels_paginated_collection = project.labels()
labels_paginated_collection

In [11]:
# Note that if you selected a `project_id` without any labels this will raise `StopIteration`
# Iterate over them to get the items out.
next(labels_paginated_collection)
# list(paginated...) should be avoided for queries that could return more than a dozen results

### Query parameters
* Query with the following conventions:
    * `DbObject.Field`

In [12]:
datasets = client.get_datasets(where=Dataset.name == DATASET_NAME)

projects = client.get_projects(
    where=((Project.name == PROJECT_NAME) &
           (Project.description == "new description field")))

# The above two queries return PaginatedCollections because the filter parameters aren't guaranteed to be unique.
# So even if there is one element returned it is in a paginatedCollection.
print(projects)
print(next(projects, None))
print(next(projects, None))
print(next(projects, None))
# We can see there is only one.

<Project {'auto_audit_number_of_labels': 3, 'auto_audit_percentage': 0.1, 'created_at': datetime.datetime(2021, 1, 20, 1, 2, 31, tzinfo=datetime.timezone.utc), 'description': 'new description field', 'last_activity_time': datetime.datetime(2021, 3, 19, 13, 46, 50, 920000, tzinfo=datetime.timezone.utc), 'name': 'Sample Project', 'setup_complete': datetime.datetime(2021, 1, 20, 1, 2, 31, 152000, tzinfo=datetime.timezone.utc), 'uid': 'ckk4q1viuc0w20704eh69u28h', 'updated_at': datetime.datetime(2021, 3, 19, 13, 46, 50, 920000, tzinfo=datetime.timezone.utc)}>
None
None


### Querying Limitations
* The DbObject used for the query must be the same as the DbObject returned by the querying function.  
* eg. is not valid since get_project returns a Project but we are filtering on a Dataset
>  `>>> projects = client.get_projects(where = Dataset.name == "dataset_name")`


### Relationship
* Relationships can be used to query for related objects (solves the limitation outlined above)
    * E.g. if a user wants all projects that have a specific dataset attached, then can run the code below.
* You can find all realtionships of a DB object in the source code
    * E.g. for a Project ( https://github.com/Labelbox/labelbox-python/blob/develop/labelbox/schema/project.py))

In [13]:
# Dataset has a Relationship to a Project so we can use the following
list(dataset.projects())
# This will return all projects that are attached to this dataset

[<Project ID: ckk4q1viuc0w107041siuht7p>]

In [14]:
sample_project_datasets = project.datasets()
list(sample_project_datasets)

[<Dataset ID: cklv1qzlv1oqn0y9ne7b9gtpb>]

### Delete
* Most DBObjects support deletion

In [15]:
#Eg.
##### project.delete()
##### dataset.delete()
##### data_row.delete()

* We reccomend using bulk operations where possible.
* You can find specific deletion instructions in tutorials on each object.